# Crowded Mayor Election
Some elections, such as the Chicago mayoral election, attract a large number of candidates.  If no candidate receives a majority of votes in the first election, then there is a run-off of the top two candidates.

Since not all voters vote, one may wonder if the two top preferred candidates will actually be chosen in the initial election.  We will explore the probability of that happening.

### Set Up Candidates
First we set the probabilities of a voter choosing each of many candidates.  Note that the top three candidates have similar probabilities.

In [42]:
# Import necessary packages
import numpy as np
from scipy import stats
from scipy.optimize import fsolve

In [33]:
PMF = np.arange(0.125,0.05,-0.01)
PMF = np.insert(PMF,[0]*2,0.13)
PMF = np.append(PMF,1-np.sum(PMF)) # probability mass function for candidates
totalProb = np.sum(PMF) # check that the sum is one
CDF = np.cumsum(PMF) # cumulative distribution function
nTrials = 10000 # number of trials 

In [32]:
np.zeros(10)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [30]:
nVotes = int(5e5) # number of votes in election
Yes = TwoBestWin(nTrials,nVotes,CDF) 

KeyboardInterrupt: 

### Chicago Election
Next we set the number of votes to be like a Chicago election.  The function *TwoBestWin* generates random election results.

In [34]:
nVotes = int(5e5) # number of votes in election
Yes = TwoBestWin(nTrials,nVotes,CDF) # when do top two candidates win the initial election
CI = binomialCI(nTrials,np.sum(Yes))
print("Chance that top two will win = ", 100*CI)

TypeError: 'float' object cannot be interpreted as an integer

### Suburb Election
Next we set the number of votes to be similar to that in a suburb. The function *TwoBestWin* generates random election results.

In [43]:
nVotes = int(5e3) # number of votes in election
Yes = TwoBestWin(nTrials,nVotes,CDF) # when do top two candidates win the initial election
CI = binomialCI(nTrials,np.sum(Yes),0.01)
print("Chance that top two will win = ", 100*CI)

Chance that top two will win =  (array([0.        , 0.00052969]), array([0., 0.]), array([0.        , 0.00052969]), array([0., 0.]), array([0.        , 0.00052969]), array([0., 0.]), array([0.        , 0.00052969]), array([0., 0.]), array([0.        , 0.00052969]), array([0., 0.]), array([0.        , 0.00052969]), array([0., 0.]), array([0.        , 0.00052969]), array([0., 0.]), array([0.        , 0.00052969]), array([0., 0.]), array([0.        , 0.00052969]), array([0., 0.]), array([0.        , 0.00052969]), array([0., 0.]), array([0.        , 0.00052969]), array([0., 0.]), array([0.        , 0.00052969]), array([0., 0.]), array([0.        , 0.00052969]), array([0., 0.]), array([0.        , 0.00052969]), array([0., 0.]), array([0.        , 0.00052969]), array([0., 0.]), array([0.        , 0.00052969]), array([0., 0.]), array([0.        , 0.00052969]), array([0., 0.]), array([0.        , 0.00052969]), array([0., 0.]), array([0.        , 0.00052969]), array([0., 0.]), array([0.        

In [46]:
np.sum(Yes)

0.0

#### TwoBestWin function

In [39]:
def TwoBestWin(nTrials,nVotes,CDF):
    Yes = np.zeros(nTrials) #initialize Yes
    nCandidate = len(CDF) #number of candidates
    vote = np.zeros(nCandidate) #initialize votes
    for ii in range(nTrials):
      votes = np.sum(np.random.rand(nVotes,1) > CDF,axis = 1) + 1 # need to debug
      for i in range(nCandidate):
         vote[i] = np.sum(votes == i)
      Yes[ii] = np.all(vote[0] > vote[2:nCandidate]) and np.all(vote[1] > vote[2:nCandidate])
    return Yes

#### binomialCI function

In [36]:
def binomialCI(ntot,nsuc,alpha):
   obsprob=nsuc/ntot # observed probability of success

   al2=alpha/2 # half of uncertainty
   if nsuc==0: # no successes observed
      plo=0  # the lower bound must be zero
   else: # use a nonlinear equation solver
      plo=fsolve(lambda x: stats.binom.cdf(nsuc-1,ntot,x)-1+al2, obsprob)[0]

   if nsuc==ntot: #no failures observed 
      pup=1 # the upper bound must be one
   else: # use a nonlinear equation solver
      pup=fsolve(lambda x: stats.binom.cdf(nsuc,ntot,x)-al2,obsprob)[0] # nonlinear equation
   
   exactci=np.array([plo,pup]) #confidence interval based on exact probabilities
   cltci= obsprob+ np.array([-1,1])*(-stats.norm.ppf(al2)*np.sqrt(obsprob*(1-obsprob))/np.sqrt(ntot))
   return exactci,cltci